# NLTK Complete Guide - Section 17: Real-World Projects

This notebook covers practical NLP projects:
1. Text Summarization
2. Keyword Extraction
3. Spam Classifier
4. Question Answering
5. Chatbot Foundation
6. Document Similarity

In [1]:
import nltk
import re
import heapq
from collections import Counter, defaultdict

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, ne_chunk

## Project 1: Extractive Text Summarization

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        TEXT SUMMARIZATION PIPELINE                       │
└─────────────────────────────────────────────────────────────────────────┘

┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│   Raw Text   │───▶│  Preprocess  │───▶│   Sentence   │───▶│    Word      │
│              │    │  (normalize) │    │  Tokenize    │    │  Tokenize    │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
                                                                   │
                                                                   ▼
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│   Summary    │◀───│  Top-N by    │◀───│   Score      │◀───│    Word      │
│   Output     │    │  Original    │    │  Sentences   │    │  Frequency   │
│              │    │  Order       │    │              │    │  (filtered)  │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
```

### How It Works

| Step | Process | Purpose |
|------|---------|---------|
| 1 | **Preprocess** | Normalize whitespace, clean text |
| 2 | **Sentence Tokenize** | Split into individual sentences |
| 3 | **Word Tokenize** | Split into words for frequency analysis |
| 4 | **Filter Stopwords** | Remove common words ("the", "is", "a") |
| 5 | **Word Frequency** | Count important words, normalize by max |
| 6 | **Score Sentences** | Sum word scores ÷ sentence length |
| 7 | **Select Top-N** | Pick highest-scoring sentences |
| 8 | **Preserve Order** | Return sentences in original order |

### Key Insight
This is **extractive** summarization - we select existing sentences, not generate new ones. The assumption: sentences with frequent important words are more central to the document's meaning.

In [2]:
class TextSummarizer:
    """Extractive text summarization using sentence scoring"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess(self, text):
        """Clean and tokenize text"""
        # Remove extra whitespace and normalize
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def score_sentences(self, text):
        """Score sentences based on word frequency"""
        text = self.preprocess(text)
        sentences = sent_tokenize(text)
        
        # Calculate word frequencies
        words = word_tokenize(text.lower())
        words = [w for w in words if w.isalpha() and w not in self.stop_words]
        word_freq = Counter(words)
        
        # Normalize frequencies
        max_freq = max(word_freq.values()) if word_freq else 1
        word_freq = {w: f/max_freq for w, f in word_freq.items()}
        
        # Score sentences
        sentence_scores = {}
        for sent in sentences:
            words = word_tokenize(sent.lower())
            score = sum(word_freq.get(w, 0) for w in words if w.isalpha())
            # Normalize by sentence length to avoid bias toward long sentences
            word_count = len([w for w in words if w.isalpha()])
            if word_count > 0:
                sentence_scores[sent] = score / word_count
        
        return sentence_scores
    
    def summarize(self, text, num_sentences=3):
        """Generate summary with top N sentences"""
        # Preprocess text FIRST, then use same preprocessed version throughout
        text = self.preprocess(text)
        sentences = sent_tokenize(text)
        
        scores = self.score_sentences(text)
        
        # Get top sentences while maintaining order
        top_sentences = heapq.nlargest(num_sentences, scores, key=scores.get)
        
        # Reorder by appearance in original text
        summary_sentences = [s for s in sentences if s in top_sentences]
        
        return ' '.join(summary_sentences)
    
    def summarize_ratio(self, text, ratio=0.3):
        """Summarize to a ratio of original length"""
        text = self.preprocess(text)
        sentences = sent_tokenize(text)
        num_sentences = max(1, int(len(sentences) * ratio))
        return self.summarize(text, num_sentences)

In [3]:
# Test the summarizer
article = """
Artificial intelligence has transformed numerous industries in recent years. 
Machine learning algorithms can now process vast amounts of data to identify patterns 
that humans might miss. Natural language processing enables computers to understand 
and generate human language. Deep learning models have achieved remarkable results in 
image recognition, speech synthesis, and game playing. Companies are investing heavily 
in AI research and development. The technology has applications in healthcare, finance, 
transportation, and entertainment. However, AI also raises ethical concerns about privacy, 
job displacement, and algorithmic bias. Researchers are working to address these challenges 
while continuing to push the boundaries of what's possible. The future of AI promises 
even more exciting developments as computing power increases and algorithms improve.
"""

summarizer = TextSummarizer()

print("ORIGINAL TEXT:")
print("=" * 60)
print(article.strip())
print(f"\n({len(sent_tokenize(article))} sentences)")

print("\n" + "=" * 60)
print("SUMMARY (3 sentences):")
print("=" * 60)
summary = summarizer.summarize(article, num_sentences=3)
print(summary)

ORIGINAL TEXT:
Artificial intelligence has transformed numerous industries in recent years. 
Machine learning algorithms can now process vast amounts of data to identify patterns 
that humans might miss. Natural language processing enables computers to understand 
and generate human language. Deep learning models have achieved remarkable results in 
image recognition, speech synthesis, and game playing. Companies are investing heavily 
in AI research and development. The technology has applications in healthcare, finance, 
transportation, and entertainment. However, AI also raises ethical concerns about privacy, 
job displacement, and algorithmic bias. Researchers are working to address these challenges 
while continuing to push the boundaries of what's possible. The future of AI promises 
even more exciting developments as computing power increases and algorithms improve.

(9 sentences)

SUMMARY (3 sentences):
Natural language processing enables computers to understand and generate hu

## Project 2: Keyword Extraction

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                       KEYWORD EXTRACTION PIPELINE                        │
└─────────────────────────────────────────────────────────────────────────┘

                              ┌─────────────────┐
                              │    Raw Text     │
                              └────────┬────────┘
                                       │
                         ┌─────────────┴─────────────┐
                         ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │  SINGLE WORDS   │         │  NOUN PHRASES   │
              │    Pipeline     │         │    Pipeline     │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │ Word Tokenize   │         │ Sent Tokenize   │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │   POS Tagging   │         │   POS Tagging   │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │ Filter: NN, JJ  │         │ Chunk: NP Grammar│
              │ (Nouns, Adj)    │         │ {<JJ>*<NN.*>+}  │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │   Lemmatize     │         │ Extract Phrases │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       ▼                           ▼
              ┌─────────────────┐         ┌─────────────────┐
              │ Count Frequency │         │ Count Frequency │
              └────────┬────────┘         └────────┬────────┘
                       │                           │
                       └─────────────┬─────────────┘
                                     ▼
                          ┌─────────────────┐
                          │   Top Keywords  │
                          │ + Top Phrases   │
                          └─────────────────┘
```

### How It Works

| Step | Process | Purpose |
|------|---------|---------|
| 1 | **POS Tagging** | Identify word types (noun, verb, adj) |
| 2 | **Filter by POS** | Keep only nouns (NN*) and adjectives (JJ) |
| 3 | **Lemmatize** | Normalize words ("learning" → "learn") |
| 4 | **Chunking** | Extract multi-word phrases using grammar |
| 5 | **Frequency Count** | Rank by occurrence count |

### Why POS Filtering?
Keywords are typically **nouns** (concepts, entities) and **adjectives** (descriptors). Verbs, prepositions, and articles rarely make good keywords.

In [4]:
class KeywordExtractor:
    """Extract keywords using TF-IDF-like scoring and POS filtering"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
    
    def extract_candidates(self, text):
        """Extract candidate keywords (nouns and noun phrases)"""
        tokens = word_tokenize(text.lower())
        tagged = pos_tag(tokens)
        
        # Filter for nouns and adjectives
        candidates = []
        for word, tag in tagged:
            if tag.startswith(('NN', 'JJ')) and word.isalpha():
                if word not in self.stop_words and len(word) > 2:
                    lemma = self.lemmatizer.lemmatize(word)
                    candidates.append(lemma)
        
        return candidates
    
    def extract_phrases(self, text):
        """Extract noun phrases as potential keywords"""
        from nltk.chunk import RegexpParser
        from nltk.tree import Tree
        
        grammar = r"NP: {<JJ>*<NN.*>+}"
        parser = RegexpParser(grammar)
        
        sentences = sent_tokenize(text)
        phrases = []
        
        for sent in sentences:
            tokens = word_tokenize(sent)
            tagged = pos_tag(tokens)
            tree = parser.parse(tagged)
            
            for subtree in tree:
                if isinstance(subtree, Tree) and subtree.label() == 'NP':
                    phrase = ' '.join(w.lower() for w, t in subtree.leaves())
                    if len(phrase.split()) > 1:  # Multi-word phrases
                        phrases.append(phrase)
        
        return phrases
    
    def extract_keywords(self, text, top_n=10, include_phrases=True):
        """Extract top keywords"""
        # Single word keywords
        candidates = self.extract_candidates(text)
        word_freq = Counter(candidates)
        
        # Score by frequency and position
        keywords = word_freq.most_common(top_n)
        
        result = {'single_words': keywords}
        
        if include_phrases:
            phrases = self.extract_phrases(text)
            phrase_freq = Counter(phrases)
            result['phrases'] = phrase_freq.most_common(top_n // 2)
        
        return result

In [5]:
extractor = KeywordExtractor()

text = """
Machine learning is a subset of artificial intelligence that enables computers 
to learn from data. Deep learning uses neural networks with multiple layers. 
Natural language processing helps machines understand human language. 
Computer vision allows systems to interpret visual information. 
Reinforcement learning trains agents through rewards and penalties.
Data science combines machine learning with statistical analysis.
Feature engineering is crucial for machine learning model performance.
"""

keywords = extractor.extract_keywords(text, top_n=10)

print("KEYWORD EXTRACTION")
print("=" * 40)

print("\nTop Single Words:")
for word, freq in keywords['single_words']:
    print(f"  {word}: {freq}")

print("\nTop Phrases:")
for phrase, freq in keywords['phrases']:
    print(f"  {phrase}: {freq}")

KEYWORD EXTRACTION

Top Single Words:
  machine: 4
  learning: 2
  computer: 2
  data: 2
  language: 2
  subset: 1
  artificial: 1
  intelligence: 1
  deep: 1
  neural: 1

Top Phrases:
  machine learning: 1
  artificial intelligence: 1
  deep learning: 1
  neural networks: 1
  multiple layers: 1


## Project 3: Spam Classifier

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         SPAM CLASSIFIER PIPELINE                         │
└─────────────────────────────────────────────────────────────────────────┘

                           TRAINING PHASE
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│  Labeled     │───▶│   Feature    │───▶│   Feature    │───▶│    Train     │
│  Messages    │    │  Extraction  │    │   Vectors    │    │ Naive Bayes  │
│ (spam/ham)   │    │              │    │              │    │   Classifier │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
                                                                   │
                                                                   ▼
                                                            ┌──────────────┐
                                                            │   Trained    │
                                                            │    Model     │
                                                            └──────────────┘
                                                                   │
                          PREDICTION PHASE                         │
┌──────────────┐    ┌──────────────┐    ┌──────────────┐           │
│    New       │───▶│   Feature    │───▶│   Classify   │◀──────────┘
│   Message    │    │  Extraction  │    │  (spam/ham)  │
└──────────────┘    └──────────────┘    └──────────────┘
                           │                   │
                           ▼                   ▼
                    ┌──────────────┐    ┌──────────────┐
                    │   Features   │    │  Probability │
                    │  • has_free  │    │   spam: 85%  │
                    │  • has_caps  │    │   ham:  15%  │
                    │  • has_url   │    │              │
                    │  • ...       │    │              │
                    └──────────────┘    └──────────────┘
```

### Feature Engineering (The Key!)

```
┌─────────────────────────────────────────────────────────────────┐
│                     FEATURE EXTRACTION                          │
├─────────────────────────────────────────────────────────────────┤
│  Input: "FREE iPhone! Click NOW to claim your prize!!!"        │
├─────────────────────────────────────────────────────────────────┤
│  Word Features:           Pattern Features:                     │
│  • has_free: True         • has_url: False                      │
│  • has_winner: False      • has_caps: True (FREE, NOW)          │
│  • has_click: True        • has_exclaim: True                   │
│  • has_urgent: False      • exclaim_count: True (>2)            │
│  • has_money: False       • has_dollar: False                   │
│                                                                 │
│  Statistics:                                                    │
│  • spam_word_count: 3 (free, click, prize)                     │
│  • short_message: True                                          │
└─────────────────────────────────────────────────────────────────┘
```

### How Naive Bayes Works

$$P(\text{spam}|\text{features}) = \frac{P(\text{features}|\text{spam}) \cdot P(\text{spam})}{P(\text{features})}$$

The classifier learns:
- How often each feature appears in spam vs ham
- Multiplies probabilities assuming feature independence (naive assumption)

In [6]:
from nltk.classify import NaiveBayesClassifier

class SpamClassifier:
    """Simple spam detection using Naive Bayes"""
    
    def __init__(self):
        self.classifier = None
        self.stop_words = set(stopwords.words('english'))
        
        # Spam indicators
        self.spam_words = {
            'free', 'winner', 'cash', 'prize', 'urgent', 'congratulations',
            'click', 'subscribe', 'offer', 'limited', 'act', 'now',
            'money', 'credit', 'loan', 'discount', 'deal', 'buy'
        }
    
    def extract_features(self, text):
        """Extract features from text"""
        text_lower = text.lower()
        words = set(word_tokenize(text_lower))
        
        features = {
            # Word presence
            'has_free': 'free' in words,
            'has_winner': 'winner' in words,
            'has_click': 'click' in words,
            'has_urgent': 'urgent' in words,
            'has_money': 'money' in words,
            
            # Patterns
            'has_url': bool(re.search(r'https?://', text_lower)),
            'has_phone': bool(re.search(r'\d{3}[-.\s]?\d{3}[-.\s]?\d{4}', text)),
            'has_email': bool(re.search(r'@\w+\.\w+', text)),
            'has_caps': bool(re.search(r'[A-Z]{3,}', text)),
            'has_exclaim': '!' in text,
            'exclaim_count': text.count('!') > 2,
            'has_dollar': '$' in text,
            
            # Statistics
            'spam_word_count': len(words & self.spam_words),
            'short_message': len(words) < 20,
        }
        
        return features
    
    def train(self, training_data):
        """Train classifier with labeled data: [(text, label), ...]"""
        featuresets = [
            (self.extract_features(text), label)
            for text, label in training_data
        ]
        self.classifier = NaiveBayesClassifier.train(featuresets)
    
    def classify(self, text):
        """Classify text as spam or ham"""
        features = self.extract_features(text)
        return self.classifier.classify(features)
    
    def probability(self, text):
        """Get spam probability"""
        features = self.extract_features(text)
        prob_dist = self.classifier.prob_classify(features)
        return {
            'spam': prob_dist.prob('spam'),
            'ham': prob_dist.prob('ham')
        }

In [7]:
# Training data
training_data = [
    # Spam examples
    ("CONGRATULATIONS! You've won $1000! Click here NOW!", "spam"),
    ("FREE MONEY! Act now for your cash prize!", "spam"),
    ("Urgent: Your account needs verification. Click link.", "spam"),
    ("Winner! Claim your free gift card today!", "spam"),
    ("Limited offer! Buy now and save 90%!", "spam"),
    ("You're selected for exclusive deal! Call 555-1234", "spam"),
    ("Get rich quick! Make $5000 daily from home!", "spam"),
    ("DISCOUNT!!! Subscribe now for FREE samples!!!", "spam"),
    
    # Ham examples
    ("Hey, are we still meeting for lunch tomorrow?", "ham"),
    ("The project deadline has been extended to Friday.", "ham"),
    ("Thanks for sending the report. I'll review it today.", "ham"),
    ("Can you pick up groceries on your way home?", "ham"),
    ("Meeting rescheduled to 3pm in conference room B.", "ham"),
    ("Happy birthday! Hope you have a great day.", "ham"),
    ("I attached the documents you requested.", "ham"),
    ("Let me know when you're free to discuss the proposal.", "ham"),
]

# Train classifier
spam_classifier = SpamClassifier()
spam_classifier.train(training_data)

print("Spam Classifier trained!")
print("\nMost informative features:")
spam_classifier.classifier.show_most_informative_features(5)

Spam Classifier trained!

Most informative features:
Most Informative Features
             has_exclaim = False             ham : spam   =      5.0 : 1.0
             has_exclaim = True             spam : ham    =      5.0 : 1.0
         spam_word_count = 0                 ham : spam   =      5.0 : 1.0
                has_free = True             spam : ham    =      2.3 : 1.0
                has_caps = False             ham : spam   =      1.5 : 1.0


In [8]:
# Test messages
test_messages = [
    "FREE iPhone! Click now to claim your prize!",
    "Don't forget about the team meeting at 2pm.",
    "URGENT! Your account will be suspended! Act NOW!",
    "Can you send me the updated spreadsheet?",
    "Winner selected! Claim $500 gift card here!",
]

print("SPAM DETECTION RESULTS")
print("=" * 60)

for msg in test_messages:
    result = spam_classifier.classify(msg)
    probs = spam_classifier.probability(msg)
    
    emoji = "🚫" if result == "spam" else "✅"
    
    print(f"\n{emoji} {result.upper()} ({probs['spam']:.1%} spam)")
    print(f"   \"{msg[:50]}...\"" if len(msg) > 50 else f"   \"{msg}\"")

SPAM DETECTION RESULTS

🚫 SPAM (99.9% spam)
   "FREE iPhone! Click now to claim your prize!"

✅ HAM (0.6% spam)
   "Don't forget about the team meeting at 2pm."

🚫 SPAM (99.8% spam)
   "URGENT! Your account will be suspended! Act NOW!"

✅ HAM (0.6% spam)
   "Can you send me the updated spreadsheet?"

🚫 SPAM (94.2% spam)
   "Winner selected! Claim $500 gift card here!"


## Project 4: Simple Question Answering

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      QUESTION ANSWERING PIPELINE                         │
└─────────────────────────────────────────────────────────────────────────┘

┌──────────────┐                              ┌──────────────┐
│   Context    │                              │   Question   │
│   Document   │                              │              │
└──────┬───────┘                              └──────┬───────┘
       │                                             │
       ▼                                             ▼
┌──────────────┐                              ┌──────────────┐
│   Sentence   │                              │  Preprocess  │
│   Tokenize   │                              │   Question   │
└──────┬───────┘                              └──────┬───────┘
       │                                             │
       ▼                                             │
┌──────────────────────────────────────────────────────────────┐
│                                                              │
│  ┌────────────┐  ┌────────────┐  ┌────────────┐             │
│  │ Sentence 1 │  │ Sentence 2 │  │ Sentence N │   ...       │
│  └─────┬──────┘  └─────┬──────┘  └─────┬──────┘             │
│        │               │               │                     │
│        ▼               ▼               ▼                     │
│  ┌────────────┐  ┌────────────┐  ┌────────────┐             │
│  │ Preprocess │  │ Preprocess │  │ Preprocess │             │
│  └─────┬──────┘  └─────┬──────┘  └─────┬──────┘             │
│        │               │               │                     │
│        └───────────────┼───────────────┘                     │
│                        ▼                                     │
│              ┌─────────────────┐◀────────────────────────────┤
│              │    Calculate    │         Question Words      │
│              │   Similarity    │                             │
│              │   (Jaccard)     │                             │
│              └────────┬────────┘                             │
│                       │                                      │
└───────────────────────┼──────────────────────────────────────┘
                        ▼
              ┌─────────────────┐
              │  Rank Sentences │
              │   by Similarity │
              └────────┬────────┘
                       │
                       ▼
              ┌─────────────────┐
              │   Best Match    │
              │    = Answer     │
              └─────────────────┘
```

### Jaccard Similarity

$$\text{Similarity}(Q, S) = \frac{|Q \cap S|}{|Q \cup S|}$$

Where:
- $Q$ = Set of words in question (after preprocessing)
- $S$ = Set of words in sentence (after preprocessing)
- $|Q \cap S|$ = Words in common
- $|Q \cup S|$ = Total unique words

### Example

```
Question: "Who created Python?"
Words: {who, created, python}

Sentence: "Python was created by Guido van Rossum..."
Words: {python, created, guido, van, rossum...}

Intersection: {python, created} = 2
Union: {who, created, python, guido, van, rossum...} = 6+

Similarity = 2/6 ≈ 0.33  ← Highest score wins!
```

In [9]:
class SimpleQA:
    """Simple extractive question answering"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess(self, text):
        """Tokenize and filter"""
        words = word_tokenize(text.lower())
        return [w for w in words if w.isalpha() and w not in self.stop_words]
    
    def similarity(self, sent1, sent2):
        """Calculate word overlap similarity"""
        words1 = set(self.preprocess(sent1))
        words2 = set(self.preprocess(sent2))
        
        if not words1 or not words2:
            return 0
        
        intersection = words1 & words2
        union = words1 | words2
        
        return len(intersection) / len(union)
    
    def find_answer(self, context, question, top_n=1):
        """Find best matching sentence(s) for the question"""
        sentences = sent_tokenize(context)
        
        # Score each sentence
        scored = []
        for sent in sentences:
            score = self.similarity(sent, question)
            scored.append((sent, score))
        
        # Sort by score
        scored.sort(key=lambda x: x[1], reverse=True)
        
        if top_n == 1:
            return scored[0][0] if scored[0][1] > 0 else "No answer found."
        
        return [s for s, score in scored[:top_n] if score > 0]

In [ ]:
qa = SimpleQA()

context = """
Python was created by Guido van Rossum and first released in 1991.
It is a high-level, interpreted programming language known for its simplicity.
Python emphasizes code readability and uses significant indentation.
The language supports multiple programming paradigms, including procedural,
object-oriented, and functional programming. Python has a large standard library
and is widely used in web development, data science, and artificial intelligence.
The name Python was inspired by Monty Python's Flying Circus.
"""

questions = [
    "Who created Python?",
    "When was Python first released?",
    "What is Python used for?",
    "Where does the name Python come from?",
]

print("QUESTION ANSWERING")
print("=" * 60)
print(f"Context: {context.strip()[:100]}...")
print("=" * 60)

for q in questions:
    answer = qa.find_answer(context, q)
    print(f"\nQ: {q}")
    print(f"A: {answer}")

QUESTION ANSWERING
Context: Python was created by Guido van Rossum and first released in 1991.
It is a high-level, interpreted p...

Q: Who invented Python?
A: The name Python was inspired by Monty Python's Flying Circus.

Q: When was Python first released?
A: 
Python was created by Guido van Rossum and first released in 1991.

Q: What is Python used for?
A: Python has a large standard library
and is widely used in web development, data science, and artificial intelligence.

Q: Where does the name Python come from?
A: The name Python was inspired by Monty Python's Flying Circus.


## Project 5: Simple Chatbot

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                          CHATBOT PIPELINE                                │
└─────────────────────────────────────────────────────────────────────────┘

                         ┌─────────────────┐
                         │   User Input    │
                         │  "Hello there!" │
                         └────────┬────────┘
                                  │
                                  ▼
                         ┌─────────────────┐
                         │   Preprocess    │
                         │  (lowercase)    │
                         │ "hello there!"  │
                         └────────┬────────┘
                                  │
                                  ▼
                    ┌─────────────────────────────┐
                    │     PATTERN MATCHING        │
                    │                             │
                    │  ┌───────────────────────┐  │
                    │  │ r'hello|hi|hey'      ├──┼──▶ Match! 
                    │  │ r'how are you'       │  │
                    │  │ r'what is your name' │  │
                    │  │ r'bye|goodbye'       │  │
                    │  │ r'thank'             │  │
                    │  │ r'weather'           │  │
                    │  │ r'help'              │  │
                    │  └───────────────────────┘  │
                    │                             │
                    └──────────────┬──────────────┘
                                   │
                    ┌──────────────┴──────────────┐
                    │                             │
              Match Found?                   No Match
                    │                             │
                    ▼                             ▼
          ┌─────────────────┐           ┌─────────────────┐
          │ Random Response │           │ Default Response│
          │  from Pattern   │           │   (fallback)    │
          │    List         │           │                 │
          └────────┬────────┘           └────────┬────────┘
                   │                             │
                   └──────────────┬──────────────┘
                                  │
                                  ▼
                         ┌─────────────────┐
                         │   Bot Response  │
                         │  "Hello! How    │
                         │   can I help?"  │
                         └─────────────────┘
```

### Pattern → Response Mapping

```
┌─────────────────────────────────────────────────────────────────┐
│  PATTERN (Regex)          │  POSSIBLE RESPONSES                 │
├───────────────────────────┼─────────────────────────────────────┤
│  r'hello|hi|hey'          │  • "Hello! How can I help you?"    │
│                           │  • "Hi there! What can I do?"       │
│                           │  • "Hey! Nice to meet you!"         │
├───────────────────────────┼─────────────────────────────────────┤
│  r'how are you'           │  • "I'm doing great, thanks!"      │
│                           │  • "I'm fine! How about you?"      │
├───────────────────────────┼─────────────────────────────────────┤
│  r'bye|goodbye|quit'      │  • "Goodbye! Have a great day!"    │
│                           │  • "See you later!"                 │
└───────────────────────────┴─────────────────────────────────────┘
```

### Limitations & Extensions

| This Chatbot | More Advanced Chatbots |
|--------------|------------------------|
| Pattern matching only | Intent classification (ML) |
| No context memory | Dialogue state tracking |
| Fixed responses | Response generation (LLM) |
| No entity extraction | Named entity recognition |

In [11]:
import random

class SimpleChatbot:
    """Pattern-matching chatbot with NLTK"""
    
    def __init__(self, name="Bot"):
        self.name = name
        self.context = {}
        
        # Response patterns
        self.patterns = {
            r'hello|hi|hey': [
                "Hello! How can I help you today?",
                "Hi there! What can I do for you?",
                "Hey! Nice to meet you!"
            ],
            r'how are you': [
                "I'm doing great, thanks for asking!",
                "I'm fine! How about you?",
                "All good here!"
            ],
            r'what is your name|who are you': [
                f"I'm {name}, your friendly chatbot!",
                f"My name is {name}. Nice to meet you!"
            ],
            r'bye|goodbye|quit|exit': [
                "Goodbye! Have a great day!",
                "See you later!",
                "Bye! Take care!"
            ],
            r'thank': [
                "You're welcome!",
                "Happy to help!",
                "No problem!"
            ],
            r'weather': [
                "I can't check the weather, but I hope it's nice!",
                "Sorry, I don't have access to weather data."
            ],
            r'help': [
                "I can chat with you! Try saying hello or asking me questions.",
                "I'm here to help! Just start a conversation."
            ],
        }
        
        self.default_responses = [
            "I'm not sure I understand. Could you rephrase that?",
            "Interesting! Tell me more.",
            "I see. What else would you like to talk about?",
            "Could you explain that differently?"
        ]
    
    def preprocess(self, text):
        """Clean input text"""
        return text.lower().strip()
    
    def match_pattern(self, text):
        """Find matching pattern"""
        for pattern, responses in self.patterns.items():
            if re.search(pattern, text):
                return random.choice(responses)
        return None
    
    def respond(self, user_input):
        """Generate response to user input"""
        text = self.preprocess(user_input)
        
        # Check for pattern match
        response = self.match_pattern(text)
        
        if response:
            return response
        
        # Default response
        return random.choice(self.default_responses)
    
    def chat(self):
        """Interactive chat loop"""
        print(f"{self.name}: Hello! I'm {self.name}. Type 'quit' to exit.")
        print("-" * 50)
        
        while True:
            user_input = input("You: ").strip()
            
            if not user_input:
                continue
            
            if re.search(r'bye|quit|exit', user_input.lower()):
                print(f"{self.name}: Goodbye!")
                break
            
            response = self.respond(user_input)
            print(f"{self.name}: {response}")

In [12]:
# Demo the chatbot (non-interactive)
bot = SimpleChatbot("NLTK-Bot")

demo_inputs = [
    "Hello!",
    "What is your name?",
    "How are you doing?",
    "What's the weather like?",
    "Tell me a joke",
    "Thank you!",
    "Goodbye!"
]

print("CHATBOT DEMO")
print("=" * 50)

for user_input in demo_inputs:
    response = bot.respond(user_input)
    print(f"You: {user_input}")
    print(f"Bot: {response}")
    print()

CHATBOT DEMO
You: Hello!
Bot: Hi there! What can I do for you?

You: What is your name?
Bot: My name is NLTK-Bot. Nice to meet you!

You: How are you doing?
Bot: All good here!

You: What's the weather like?
Bot: I can't check the weather, but I hope it's nice!

You: Tell me a joke
Bot: I'm not sure I understand. Could you rephrase that?

You: Thank you!
Bot: Happy to help!

You: Goodbye!
Bot: See you later!



## Project 6: Document Similarity

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     DOCUMENT SIMILARITY PIPELINE                         │
└─────────────────────────────────────────────────────────────────────────┘

                              FITTING PHASE
┌──────────────────────────────────────────────────────────────────────────┐
│                                                                          │
│  ┌────────┐  ┌────────┐  ┌────────┐                                     │
│  │  Doc 1 │  │  Doc 2 │  │  Doc N │  ...                                │
│  └───┬────┘  └───┬────┘  └───┬────┘                                     │
│      │           │           │                                           │
│      └───────────┼───────────┘                                           │
│                  ▼                                                       │
│         ┌─────────────────┐                                             │
│         │   Preprocess    │  (tokenize, lowercase, lemmatize,           │
│         │   All Docs      │   remove stopwords)                         │
│         └────────┬────────┘                                             │
│                  │                                                       │
│                  ▼                                                       │
│         ┌─────────────────┐                                             │
│         │ Build Vocabulary│  {machine, learning, ai, neural, ...}       │
│         └────────┬────────┘                                             │
│                  │                                                       │
│                  ▼                                                       │
│         ┌─────────────────┐                                             │
│         │  Calculate IDF  │  IDF(word) = log(N / doc_count(word))       │
│         │  for each word  │                                             │
│         └─────────────────┘                                             │
│                                                                          │
└──────────────────────────────────────────────────────────────────────────┘

                           SIMILARITY PHASE
┌──────────────────────────────────────────────────────────────────────────┐
│                                                                          │
│  ┌────────┐              ┌────────┐                                     │
│  │  Doc A │              │  Doc B │                                     │
│  └───┬────┘              └───┬────┘                                     │
│      │                       │                                           │
│      ▼                       ▼                                           │
│  ┌────────────┐         ┌────────────┐                                  │
│  │  TF-IDF    │         │  TF-IDF    │                                  │
│  │  Vector A  │         │  Vector B  │                                  │
│  └─────┬──────┘         └─────┬──────┘                                  │
│        │                      │                                          │
│        └──────────┬───────────┘                                          │
│                   ▼                                                      │
│          ┌─────────────────┐                                            │
│          │     Cosine      │                                            │
│          │   Similarity    │                                            │
│          └────────┬────────┘                                            │
│                   │                                                      │
│                   ▼                                                      │
│          ┌─────────────────┐                                            │
│          │  Score: 0.75    │  (0 = unrelated, 1 = identical)            │
│          └─────────────────┘                                            │
│                                                                          │
└──────────────────────────────────────────────────────────────────────────┘
```

### TF-IDF Calculation

**TF (Term Frequency)**: How often a word appears in THIS document
$$TF(t, d) = \frac{\text{count of } t \text{ in } d}{\text{max count in } d}$$

**IDF (Inverse Document Frequency)**: How rare a word is across ALL documents
$$IDF(t) = \log\frac{N}{\text{documents containing } t}$$

**TF-IDF**: 
$$\text{TF-IDF}(t, d) = TF(t, d) \times IDF(t)$$

### Cosine Similarity

$$\cos(\theta) = \frac{\vec{A} \cdot \vec{B}}{||\vec{A}|| \times ||\vec{B}||} = \frac{\sum A_i B_i}{\sqrt{\sum A_i^2} \times \sqrt{\sum B_i^2}}$$

### Example

```
Doc1: "machine learning is great"     → TF-IDF: [0.5, 0.5, 0.1, 0.3, ...]
Doc2: "deep learning uses neural"     → TF-IDF: [0.0, 0.5, 0.0, 0.0, ...]
                                              ↑
                                         "learning" in both

Cosine Similarity = high where vectors point same direction
```

In [13]:
import math

class DocumentSimilarity:
    """Calculate document similarity using TF-IDF and cosine similarity"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.documents = []
        self.vocabulary = set()
        self.idf = {}
    
    def preprocess(self, text):
        """Tokenize and normalize text"""
        tokens = word_tokenize(text.lower())
        return [
            self.lemmatizer.lemmatize(w)
            for w in tokens
            if w.isalpha() and w not in self.stop_words
        ]
    
    def fit(self, documents):
        """Fit on a collection of documents"""
        self.documents = [self.preprocess(doc) for doc in documents]
        
        # Build vocabulary and calculate IDF
        doc_count = len(self.documents)
        word_doc_count = defaultdict(int)
        
        for doc in self.documents:
            for word in set(doc):
                word_doc_count[word] += 1
                self.vocabulary.add(word)
        
        # Calculate IDF
        for word, count in word_doc_count.items():
            self.idf[word] = math.log(doc_count / count)
    
    def tf_idf(self, document):
        """Calculate TF-IDF vector for a document"""
        if isinstance(document, str):
            document = self.preprocess(document)
        
        word_count = Counter(document)
        max_count = max(word_count.values()) if word_count else 1
        
        vector = {}
        for word in self.vocabulary:
            tf = word_count.get(word, 0) / max_count
            idf = self.idf.get(word, 0)
            vector[word] = tf * idf
        
        return vector
    
    def cosine_similarity(self, vec1, vec2):
        """Calculate cosine similarity between two vectors"""
        dot_product = sum(vec1.get(k, 0) * vec2.get(k, 0) for k in self.vocabulary)
        
        mag1 = math.sqrt(sum(v ** 2 for v in vec1.values()))
        mag2 = math.sqrt(sum(v ** 2 for v in vec2.values()))
        
        if mag1 == 0 or mag2 == 0:
            return 0
        
        return dot_product / (mag1 * mag2)
    
    def similarity_matrix(self, documents):
        """Calculate pairwise similarity matrix"""
        self.fit(documents)
        vectors = [self.tf_idf(doc) for doc in self.documents]
        
        n = len(documents)
        matrix = [[0] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                matrix[i][j] = self.cosine_similarity(vectors[i], vectors[j])
        
        return matrix
    
    def find_similar(self, query, documents, top_n=3):
        """Find most similar documents to a query"""
        self.fit(documents)
        query_vector = self.tf_idf(query)
        
        similarities = []
        for i, doc in enumerate(self.documents):
            doc_vector = self.tf_idf(doc)
            sim = self.cosine_similarity(query_vector, doc_vector)
            similarities.append((i, sim))
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]

In [14]:
# Test document similarity
documents = [
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with multiple layers.",
    "Natural language processing helps computers understand text.",
    "Python is widely used for machine learning and AI.",
    "Data science combines statistics with programming.",
]

sim_calc = DocumentSimilarity()
matrix = sim_calc.similarity_matrix(documents)

print("DOCUMENT SIMILARITY MATRIX")
print("=" * 50)

# Print matrix
print("\n     ", end="")
for i in range(len(documents)):
    print(f"Doc{i+1:>6}", end="")
print()

for i, row in enumerate(matrix):
    print(f"Doc{i+1}", end="")
    for val in row:
        print(f"{val:>7.2f}", end="")
    print()

print("\n" + "=" * 50)
print("Documents:")
for i, doc in enumerate(documents):
    print(f"Doc{i+1}: {doc[:50]}...")

DOCUMENT SIMILARITY MATRIX

     Doc     1Doc     2Doc     3Doc     4Doc     5
Doc1   1.00   0.02   0.00   0.11   0.00
Doc2   0.02   1.00   0.00   0.02   0.00
Doc3   0.00   0.00   1.00   0.00   0.00
Doc4   0.11   0.02   0.00   1.00   0.00
Doc5   0.00   0.00   0.00   0.00   1.00

Documents:
Doc1: Machine learning is a subset of artificial intelli...
Doc2: Deep learning uses neural networks with multiple l...
Doc3: Natural language processing helps computers unders...
Doc4: Python is widely used for machine learning and AI....
Doc5: Data science combines statistics with programming....


In [15]:
# Find similar documents to a query
query = "How does artificial intelligence work?"

similar = sim_calc.find_similar(query, documents, top_n=3)

print(f"\nQuery: '{query}'")
print("\nMost similar documents:")
for idx, score in similar:
    print(f"  {score:.2f}: {documents[idx]}")


Query: 'How does artificial intelligence work?'

Most similar documents:
  0.76: Machine learning is a subset of artificial intelligence.
  0.00: Deep learning uses neural networks with multiple layers.
  0.00: Natural language processing helps computers understand text.


## Summary

### Projects Covered

| Project | Key Techniques |
|---------|----------------|
| Text Summarization | Sentence scoring, word frequency |
| Keyword Extraction | TF-IDF, POS filtering, noun phrases |
| Spam Classifier | Naive Bayes, feature extraction |
| Question Answering | Sentence similarity, extractive QA |
| Chatbot | Pattern matching, regex |
| Document Similarity | TF-IDF, cosine similarity |

### Key Takeaways
- Combine multiple NLTK tools for real applications
- Preprocessing is crucial for all NLP tasks
- Feature engineering impacts model performance
- Start simple, then add complexity as needed